In [ ]:
%%capture
!pip install pygame
!pip install pygad

!git clone https://github.com/karinemiras/evoman_framework.git tmp
!cp -r /kaggle/working/tmp/* /kaggle/working/
!rm -R /kaggle/working/tmp

In [ ]:
import sys, os
import numpy as np
import pandas as pd

import random

from evoman.environment import Environment
from evoman.controller import Controller
from demo_controller import player_controller
from deap import base, creator, tools, algorithms
import multiprocessing
import optuna

import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
class individual:
    
    def __init__(self, initial_range, individual_size):
        self.genes = np.random.uniform(low=-initial_range,high=initial_range,size=(individual_size))
        self.evaluated = False
        self.fitness = np.zeros((8))
        self.player_energy = np.zeros((8))
        self.enemy_energy = np.zeros((8))
        
class log_book():
    
    def __init__(self):
        self.logs = []
        
    def create_log(self,pop,gen,nevals,train_enemies,hof):
        log = {}
        train_gains = np.array([sum(ind.gains[train_enemies-1]) for ind in pop])
        all_gains = np.array([ind.sum_gain for ind in pop])
        all_defeated = np.array([ind.defeated for ind in pop])
        log['gen'] = gen
        log['nevals'] = nevals
        log['train_enemies_gain_mean'] = train_gains.mean()
        log['train_enemies_gain_max'] = train_gains.max()
        log['train_enemies_gain_std'] = train_gains.std()
        log['all_enemies_gain_mean'] = all_gains.mean()
        log['all_enemies_gain_max'] = all_gains.max()
        log['all_enemies_defeated_max'] = all_defeated.max()
        log['hof_max'] = hof['max_value']
        self.logs.append(log)
        self.pd = pd.DataFrame(self.logs)
        
def play(x):
    global env
    if x.evaluated:
        return x
    else:
        for enemy in range(1,9):
            env.update_parameter("enemies",[enemy])
            res = env.play(x.genes)
            x.fitness[enemy-1] = res[0]
            x.player_energy[enemy-1] = res[1]
            x.enemy_energy[enemy-1] = res[2]
        x.gains = (x.player_energy-x.enemy_energy)
        x.sum_gain = x.gains.sum()
        x.defeated = (x.enemy_energy==0).sum()
        x.evaluated = True
        return x

In [ ]:
def parameter_count(hidden_neurons):
    if hidden_neurons>0:
        n_w = (20*hidden_neurons) + (hidden_neurons*5)
        n_b = hidden_neurons + 5
        return n_w + n_b
    else:
        return (20*5)+5

In [ ]:
def cxWholeAritmetic(parent1,parent2,alpha):
    child1,child2 = individual(0,len(parent1.genes)), individual(0,len(parent1.genes))
    child1.genes = (alpha*parent1.genes)+((1-alpha)*parent2.genes)
    child2.genes = (alpha*parent2.genes)+((1-alpha)*parent1.genes)
    return child1,child2

def cxBlend(parent1,parent2,alpha):
    child1,child2 = individual(0,len(parent1.genes)), individual(0,len(parent1.genes))
    gamma = (1+2*alpha)*np.random.uniform(size=(len(parent1.genes,))) - alpha
    child1.genes = (gamma*parent1.genes)+((1-gamma)*parent2.genes)
    child2.genes = (gamma*parent2.genes)+((1-gamma)*parent1.genes)
    return child1, child2

In [ ]:
def mutGaussian(ind,sigma,indpb):
    new_ind = individual(0,len(ind.genes))
    mutation = [np.random.normal(0,sigma) if np.random.uniform()<indpb else 0 for i in range(len(ind.genes))]
    new_ind.genes = ind.genes + np.array(mutation)
    return new_ind

def mutGaussianAdaptive(ind,sigma,tau,indpb):
    new_ind = individual(0,len(ind.genes))
    new_sigma = sigma * np.exp(np.random.normal(0,tau,size=(len(ind.genes),)))
    new_sigma[new_sigma<0.1] = 0.1
    mutation = [np.random.normal(0,s) if np.random.uniform()<indpb else 0 for s in new_sigma]
    new_ind.genes = ind.genes + np.array(mutation)
    return new_ind

def selTournament(individuals,k,tourn_size,train_enemies):
    selected = []
    for i in range(k):
        tourn_selection = np.random.choice(individuals,tourn_size)
        best = np.argmax([ind.gains[train_enemies-1].sum() for ind in tourn_selection])
        selected.append(tourn_selection[best])
    return selected

In [ ]:
def genetic_algorithm(train_enemies,initial_range,pop_size,n_offspring,tournsize_parent,cxpb,cx_type,
                      alpha,mutpb,mut_type,sigma,indpb,tau,tournsize_survival,ngen,verbose):
    
    global individual_size,env

    train_enemies = np.array(train_enemies)
    pop = [individual(initial_range,individual_size) for i in range(pop_size)]
    hof = {'best_solution':-np.inf,'max_value':-np.inf}
    logbook = log_book()

    with multiprocessing.Pool() as pool:
        # evaluate initial population
        pop = list(pool.map(play,pop))

        # record best solution
        for ind in pop:
            if ind.sum_gain>hof['max_value']:
                hof['best_solution'] = ind
                hof['max_value'] = ind.sum_gain
        
        logbook.create_log(pop,0,len(pop),train_enemies,hof)
        #print("gen: 0")

        for g in range(ngen):
            # select parents
            parents = selTournament(individuals=pop,k=n_offspring*pop_size,tourn_size=tournsize_parent,train_enemies=train_enemies)

            # create offspring
            offspring = []
            for child1, child2 in zip(parents[::2], parents[1::2]):
                if np.random.uniform() < cxpb:
                    if cx_type=="whole_aritmetic":
                        if alpha=="random":
                            alpha=np.random.uniform()
                            child1,child2 = cxWholeAritmetic(child1, child2, alpha)
                        else:
                            child1,child2 = cxWholeAritmetic(child1, child2, alpha)
                    elif cx_type=="blend":
                        if alpha=="random":
                            alpha=np.random.uniform()
                            child1,child2 = cxBlend(child1, child2, alpha)
                        else:
                            child1,child2 = cxBlend(child1, child2, alpha)
                offspring.append(child1)
                offspring.append(child2)

            # apply mutation to offspring
            for i in range(len(offspring)):
                if np.random.uniform() < mutpb:
                    if mut_type=="normal":
                        if indpb=="random":
                            indpb=np.random.uniform()
                            offspring[i] = mutGaussian(offspring[i],sigma,indpb)
                        else:
                            offspring[i] = mutGaussian(offspring[i],sigma,indpb)
                    elif mut_type=="adaptive":
                        if indpb=="random":
                            indpb=np.random.uniform()
                            offspring[i] = mutGaussianAdaptive(offspring[i],sigma,indpb,tau)
                        else:
                            offspring[i] = mutGaussianAdaptive(offspring[i],sigma,indpb,tau)        

            # evaluate offspring
            invalid_ind = [ind for ind in offspring if not ind.evaluated]
            offspring = list(pool.map(play,offspring))

            # record best solution
            for ind in offspring:
                if ind.sum_gain>hof['max_value']:
                    hof['best_solution'] = ind
                    hof['max_value'] = ind.sum_gain

            # select the next generation of the population
            pop = selTournament(individuals=offspring,k=pop_size,tourn_size=tournsize_survival,train_enemies=train_enemies)

            logbook.create_log(pop,g+1,len(invalid_ind),train_enemies,hof)
            #print("gen: "+str(g+1))

        display(logbook.pd)
        return hof,logbook
    

In [ ]:
hidden_neurons = 10
individual_size = parameter_count(hidden_neurons)

headless = True
if headless:
    os.environ["SDL_VIDEODRIVER"] = "dummy"

env = Environment(playermode="ai",
                  player_controller=player_controller(hidden_neurons),
                  speed="fastest",
                  enemymode="static",
                  level=2,
                  visuals=False,
                  logs="off")

In [ ]:
def objective(trial):
    
    params = {'train_enemies':trial.suggest_categorical("train_enemies",[[7,8],[2,5,6],[1,2,6,7]])
              'initial_range':trial.suggest_int("initial_range", 1,3,step=1),
              'pop_size':trial.suggest_int("pop_size", 50,50,step=50),
              'n_offspring':trial.suggest_int("n_offspring", 1,5,step=1),
              'tournsize_parent':trial.suggest_int("tournsize_parent", 1,10,step=1),
              'cxpb':trial.suggest_float("cxpb", 0.1, 1., step=0.1),
              'cx_type':trial.suggest_categorical("cx_type",["whole_aritmetic","blend"]),
              'alpha':trial.suggest_categorical("alpha",np.arange(0.1,1.1,0.1).tolist()+["random"]),
              'mutpb':trial.suggest_float("mutpb", 0.1, 1., step=0.1),
              'mut_type':trial.suggest_categorical("mut_type",["normal","adaptive"]),
              'sigma':trial.suggest_float("sigma", 0.5, 3, step=0.5),
              'indpb':trial.suggest_categorical("indpb",np.arange(0.1,1.1,0.1).tolist()+["random"]),
              'tau':trial.suggest_float("tau", 0.1, 1., step=0.1),
              'tournsize_survival':trial.suggest_int("tournsize_survival", 1,10,step=1),
              'ngen':trial.suggest_int("ngen", 30,30,step=30),
              'verbose':trial.suggest_int("verbose", 1,1,step=1)}
    
    global hof_tuning
    iteration_logs = []
    for i in range(3):
        hof,logs = genetic_algorithm(**params)
        iteration_logs.append(logs.pd)
        hof_tuning.append(hof)
    
    return pd.concat(iteration_logs)['all_enemies_gain_max'].mean()


In [ ]:
hof_tuning = []

study = optuna.create_study(study_name="GA", direction="maximize")

study.optimize(objective, n_trials=100)

In [ ]:
"""
params = {'train_enemies':[7,8],
          'initial_range':1,
          'pop_size':50,
          'n_offspring':2,
          'tournsize_parent':5,
          'cxpb':0.7,
          'cx_type':'blend',
          'alpha':'random',
          'mutpb':0.3,
          'mut_type':'adaptive',
          'sigma':2,
          'indpb':0.8,
          'tau':0.3,
          'tournsize_survival':10,
          'ngen':10,
          'verbose':1}
          
""""